In [ ]:
# default_exp models.plmodules

In [ ]:
# hide
%load_ext autoreload
%autoreload 2self.log("loss_1", loss1, on_step=False, on_epoch=True)

# PytorchLightning Modules
> Modules implementing <a href=\"https://pytorch-lightning.readthedocs.io/en/0.6.0/lightning-module.html\">LightningModule</a> interface.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
# export
import pytorch_lightning as pl
from pytorch_lightning.metrics import ConfusionMatrix, ROC
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.profiler import AdvancedProfiler, SimpleProfiler
from torch.optim import Optimizer
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR, ReduceLROnPlateau
from torch.utils.data import Dataset

from grade_classif.core import ifnone
from grade_classif.data.color import ColorConverter, rgb_to_lab
from grade_classif.imports import *
from grade_classif.models.losses import BCE, FocalLoss
from grade_classif.models.metrics import pcc, ssim, ClassifMetrics
from grade_classif.models.modules import *
from grade_classif.models.utils import (
    gaussian_mask,
    get_num_features,
    get_sizes,
    named_leaf_modules
)
from timm.models.vision_transformer import Block
from timm.models.layers.weight_init import trunc_normal_
from sklearn.metrics import auc
from pytorch_lightning.tuner.lr_finder import lr_find
from pytorch_lightning.trainer.states import RunningStage

In [ ]:
# export
def _get_loss(
    loss_name: str,
    weights: Sequence[float],
    reduction: str,
    device: str = "cpu",
    nc: int = 2,
) -> Callable[[torch.Tensor, torch.Tensor], torch.Tensor]:
    if weights is not None:
        weights = torch.tensor(weights, device=device)
    if loss_name == "cross-entropy":
        loss = nn.CrossEntropyLoss(
            weights,
            reduction=reduction,
        )
    elif loss_name == "bce":
        loss = BCE(
            reduction=reduction, pos_weight=weights
        )
    elif loss_name == "mse":
        loss = nn.MSELoss(reduction=reduction)
    elif loss_name == "focal":
        loss = FocalLoss(reduction=reduction)
    return loss.__call__

In [ ]:
# export
def _get_scheduler(
    opt: Optimizer, name: str, total_steps: int, lr: float
) -> torch.optim.lr_scheduler._LRScheduler:
    if name == "one-cycle":
        sched = {
            "scheduler": OneCycleLR(opt, lr, total_steps=total_steps),
            "interval": "step",
        }
    elif name == "cosine-anneal":
        sched = {"scheduler": CosineAnnealingLR(opt, total_steps), "interval": "step"}
    elif name == "reduce-on-plateau":
        sched = {
            "scheduler": ReduceLROnPlateau(opt, patience=3),
            "interval": "epoch",
            "monitor": "val_loss",
        }
    else:
        return None
    return sched

In [ ]:
# export
def _get_base_model(model_name: str, pretrained: bool = True, size=299, **kwargs):
    if "cbr" in model_name:
        args = map(int, model_name.split("_")[1:])
        base_model = CBR(*args, **kwargs)
        cut = -3
    elif "sasa" in model_name:
        args = map(int, model_name.split("_")[1:])
        base_model = SASA(*args, **kwargs)
        cut = -3
    elif "sanet" in model_name:
        splits = model_name.split("_")
        kernel_size = int(splits[-1])
        base_model = globals()[splits[0]](kernel_size, **kwargs)
        cut = -2
    elif "vit" in model_name:
        splits = model_name.split("_")
        base_model = VisionTransformer(
            img_size=size,
            patch_size=splits[1],
            embed_dim=splits[2],
            depth=splits[3],
            num_heads=splits[4],
            qkv_bias=True,
        )
        cut = -2
    else:
        base_model = timm.create_model(model_name, pretrained=pretrained, **kwargs)
        cut = -2
    base_model = nn.Sequential(*list(base_model.children())[:cut])
    return base_model

In [ ]:
# export
def _get_head(n_features: int, n_classes: int, dropout: float = 0.5):
    head = [nn.AdaptiveAvgPool2d(1), nn.Flatten()]
    head += bn_drop_lin(n_features, n_features, p=dropout / 2) + bn_drop_lin(
        n_features, n_classes, p=dropout
    )
    head = nn.Sequential(*head)
    return head

In [ ]:
# export
class BaseModule(pl.LightningModule):
    """"""

    def __init__(
        self,
        size: int = 299,
        gpus: Optional[List[int]] = None,
        model: Optional[str] = None,
        normalizer: Optional[str] = None,
        savedir: Optional[Union[Path, str]] = None,
        level: int = None,
        resume_version: Optional[str] = None,
        resume_checkpoint: Optional[int] = None,
        sample_mode: int = 0,
        weights: Optional[Sequence[float]] = None,
        epochs: int = 5,
        loss: str = "mse",
        reduction: str = "mean",
        lr: float = 1e-3,
        wd: float = 0.01,
        train_percent: float = 1.0,
        sched: Optional[str] = None,
        rand_weights: bool = False,
        dropout: float = 0.5,
        version: Optional[str] = None,
        open_mode: str = "RGB",
        mixed_precision: bool = False,
        auto_lr_find: bool = False,
        **kwargs,
    ):
        super().__init__()
        if model is None and normalizer is None:
            raise AttributeError(
                "at least one of model and normalizer should ne specified"
            )
        self.main_device = "cpu" if gpus is None else f"cuda:{gpus[0]}"
        # self.main_device = 'cuda:1'
        model_type = "normalizer" if isinstance(self, Normalizer) else "classifier"
        savedir = ifnone(savedir, Path.cwd() / "log")
        level = ifnone(level, -1)
        self.save_path = Path(savedir) / f"level_{level}"
        self.save_path = self.save_path / f"{model_type}/{model}"
        self.example_inputs = torch.rand(
            2, 3, size, size, dtype=torch.float, device=self.main_device
        )
        self.color_converter = ColorConverter(open_mode)
        self.callbacks = []
        self.save_hyperparameters()

    def post_init(self):
        leaf_modules = named_leaf_modules(self)
        size = self.hparams.size
        self.sizes, self.leaf_modules = get_sizes(
            self, input_shape=(3, size, size), leaf_modules=leaf_modules
        )
        self = self.to(self.main_device)
        if self.hparams.resume_version is not None:
            self.load(
                self.hparams.resume_version, ckpt_epoch=self.hparams.resume_checkpoint
            )

    def on_train_start(self):
        self.train()

    def training_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_nb: int
    ) -> Dict[str, torch.Tensor]:
        # REQUIRED
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        lr = self.opt.param_groups[-1]["lr"]
        log = {"train_loss": loss.detach(), "learning_rate": lr}
        self.log_dict(log, on_step=True, on_epoch=False)
        return {"loss": loss}

    def validation_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_nb: int
    ) -> Dict[str, torch.Tensor]:
        # OPTIONAL
        x, y = batch
        y_hat = self(x).detach()
        loss = self.loss(y_hat, y)
        self.log("val_loss", loss)
        return {"loss": loss, "preds": y_hat.cpu(), "labels": y.cpu()}

    def validation_epoch_end(self, outputs: List[Dict[str, torch.Tensor]]):
        # OPTIONAL
        val_loss = torch.mean(torch.stack([output["loss"] for output in outputs]))
        self.log("val_loss", val_loss)

    def test_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_nb: int
    ) -> Dict[str, torch.Tensor]:
        # OPTIONAL
        return self.validation_step(batch, batch_nb)

    def on_fit_end(self):
        try:
            path = self.trainer.checkpoint_callbacks[0].best_model_path
        except IndexError:
            print("No checkpoint callback found, skipping export.")
            return
        checkpoint = torch.load(path, map_location=self.main_device)
        self.load_state_dict(checkpoint["state_dict"])
        self = self.to(self.main_device)
        self.export_to_torchscript()

    def configure_optimizers(self) -> Union[Optimizer, Dict[str, Any]]:
        # REQUIRED
        hparams = self.hparams
        if not hasattr(self, "loss"):
            self.loss = _get_loss(
                hparams.loss,
                hparams.weights,
                hparams.reduction,
                device=self.main_device,
            )
        self.lr = hparams.lr
        self.wd = hparams.wd
        self.opt = torch.optim.AdamW(
            self.parameters(), lr=self.lr, weight_decay=self.wd
        )
        n_train_dl = len(self.trainer.datamodule.train_dataloader())
        n_iter = int(hparams.epochs * n_train_dl)
        sched = _get_scheduler(self.opt, hparams.sched, n_iter, self.lr)
        if sched is None:
            return self.opt
        else:
            self.sched = sched["scheduler"]
            return {"optimizer": self.opt, "lr_scheduler": sched}

    """def on_after_backward(self):
        for pg in self.opt.param_groups:
            for p in pg["params"]:
                p.data.mul_(1 - self.wd * pg["lr"])"""

    def load(self, version: str, ckpt_epoch: int = None):
        """
        Load a specific `version` of current model, stored in
        `self.save_path/lightning_logs`. If multiple checkpoints have been
        stored, `ckpt_epoch` can be specified to load a specific epoch. Else
        the latest epoch is loaded.
        """
        save_dir = self.save_path / f"lightning_logs/version_{version}/checkpoints"
        path = (
            list(save_dir.iterdir())[-1]
            if ckpt_epoch is None
            else save_dir / f"_ckpt_epoch_{ckpt_epoch}.ckpt"
        )
        if not path.exists():
            path = save_dir / f"epoch={ckpt_epoch}.ckpt"
        checkpoint = torch.load(path, map_location=self.main_device)
        missing, unexpected = self.load_state_dict(
            checkpoint["state_dict"], strict=False
        )
        if len(missing):
            message = f"Warning, missing keys in state dict :\n{'; '.join(missing)}"
            warnings.warn(message)
        if len(unexpected):
            message = (
                f"Warning, unexpected keys in state dict :\n{'; '.join(unexpected)}"
            )
            warnings.warn(message)
        self.version = version

    def my_summarize(self) -> pd.DataFrame:
        """
        Get a DataFrame containing the list of all leaf modules of current
        model, with their corresponding output shape.
        """
        named_modules = list(map(lambda x: x.name, self.leaf_modules))
        summary = pd.DataFrame({"Name": named_modules, "Output Shape": self.sizes})
        return summary
    
    def on_save_checkpoint(self, checkpoint: Dict[str, Any]):
        dirpath = Path(self.ckpt_callback.dirpath).parent
        self.export_to_torchscript(file_path=dirpath/f"{self.current_epoch}.pt")

    def fit(self, dm: pl.LightningDataModule, monitor="val_loss", **kwargs):
        """
        Fit the model using parameters stored in `hparams`.
        """
        logger = CometLogger(
            api_key=os.environ["COMET_API_KEY"],
            workspace="schwobr",
            save_dir=self.save_path,
            project_name="grade-classif",
            auto_metric_logging=False,
        )
        logger.experiment.add_tag("norm" if isinstance(self, Normalizer) else "classif")
        ckpt_path = (
            self.save_path
            / "lightning_logs"
            / f"version_{logger.version}"
            / "checkpoints"
        )
        if not ckpt_path.is_dir():
            ckpt_path.mkdir(parents=True)
        mode = "min" if "loss" in monitor or "err" in monitor else "max"
        self.ckpt_callback = ModelCheckpoint(
            dirpath=ckpt_path,
            save_top_k=3,
            monitor=monitor,
            save_last=True,
            mode=mode,
            filename="{epoch}",
        )
        profiler = None
        if self.hparams.profiler == "simple":
            profiler = SimpleProfiler(ckpt_path.parent / "profiler_logs.txt")
        elif self.hparams.profiler == "advanced":
            profiler = AdvancedProfiler(ckpt_path.parent / "profiler_logs.txt")
        dm.setup()
        precision = 16 if self.hparams.mixed_precision or self.hparams.deepspeed else 32
        plugins = "deepspeed" if self.hparams.deepspeed else None
        callbacks = self.callbacks + [self.ckpt_callback]
        n = len(dm.train_dataloader())
        log_every_n_steps = min(50, max(1, n // 10))
        trainer = pl.Trainer(
            gpus=self.hparams.gpus,
            callbacks=callbacks,
            logger=logger,
            precision=precision,
            min_epochs=self.hparams.epochs,
            max_epochs=self.hparams.epochs,
            plugins=plugins,
            log_every_n_steps=log_every_n_steps,
            profiler=profiler,
            weights_summary=None,
            **kwargs,
        )
        if self.hparams.auto_lr_find:
            lr_finder = lr_find(trainer, self, datamodule=dm)
            fig = lr_finder.plot(suggest=True)
            lr = lr_finder.suggestion()
            print(f"Suggested learning rate is {lr}.")
            self.hparams.lr = lr
            self.logger.experiment.log_parameter("lr", lr)
            trainer.logger.experiment.log_figure(figure_name="LR finder", figure=fig)
        self.version = trainer.logger.version
        trainer.fit(self, dm)

    def predict(self, x: torch.Tensor) -> torch.Tensor:
        """
        Make a prediction on batch `x`.
        """
        return self.eval()(x)

    def export_to_torchscript(self, file_path: Optional[str] = None):
        save_dir = self.save_path / f"lightning_logs/version_{self.version}"
        file_path = ifnone(file_path, save_dir / "best_model.pt")
        self.to_torchscript(
            file_path=str(file_path),
            method="trace",
            example_inputs=self.example_inputs,
        )

    def _create_normalizer(self):
        hparams = self.hparams
        if hparams.normalizer is not None:
            norm = torch.jit.load(hparams.normalizer, map_location="cpu")
            for p in norm.parameters():
                p.requires_grad = False
            norm = norm.eval()
            self.norm = norm

Base class for all classes implementing [`LightningModule`](https://pytorch-lightning.readthedocs.io/en/0.6.0/lightning-module.html) interface. Defines a lot of convenience function to be used by children classes.

In [ ]:
show_doc(BaseModule.load)

<h4 id="BaseModule.load" class="doc_header"><code>BaseModule.load</code><a href="__main__.py#L139" class="source_link" style="float:right">[source]</a></h4>

> <code>BaseModule.load</code>(**`version`**:`str`, **`ckpt_epoch`**:`int`=*`None`*)

Load a specific `version` of current model, stored in
`self.save_path/lightning_logs`. If multiple checkpoints have been
stored, `ckpt_epoch` can be specified to load a specific epoch. Else
the latest epoch is loaded.

In [ ]:
show_doc(BaseModule.my_summarize)

<h4 id="BaseModule.my_summarize" class="doc_header"><code>BaseModule.my_summarize</code><a href="__main__.py#L168" class="source_link" style="float:right">[source]</a></h4>

> <code>BaseModule.my_summarize</code>()

Get a DataFrame containing the list of all leaf modules of current
model, with their corresponding output shape.

In [ ]:
show_doc(BaseModule.fit)

<h4 id="BaseModule.fit" class="doc_header"><code>BaseModule.fit</code><a href="__main__.py#L177" class="source_link" style="float:right">[source]</a></h4>

> <code>BaseModule.fit</code>(**`dm`**:`LightningDataModule`, **`monitor`**=*`'val_loss'`*, **\*\*`kwargs`**)

Fit the model using parameters stored in [`hparams`](/grade_classif/params.parser.html#hparams).

In [ ]:
show_doc(BaseModule.predict)

<h4 id="BaseModule.predict" class="doc_header"><code>BaseModule.predict</code><a href="__main__.py#L239" class="source_link" style="float:right">[source]</a></h4>

> <code>BaseModule.predict</code>(**`x`**:`Tensor`)

Make a prediction on batch `x`.

In [ ]:
# export
def _get_keys_in_list_and_apply(
    list_of_dicts: Sequence[Dict[str, Any]],
    *keys,
    apply_func: Optional[Callable[[Any], Any]] = None
) -> List[List[Any]]:
    res = []
    for k in keys:
        sub_list = []
        for d in list_of_dicts:
            sub_list.append(d[k])
        if apply_func is not None:
            sub_list = apply_func(sub_list)
        res.append(sub_list)
    return res

In [ ]:
# export
class Normalizer(BaseModule):
    """"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        input_shape = (3, self.hparams.size, self.hparams.size)
        self.unet = DynamicUnet(
            self.hparams.model,
            n_classes=3,
            input_shape=input_shape,
            pretrained=not self.hparams.rand_weights,
        )
        self.post_init()
    
    def on_fit_start(self):
        x, y = next(iter(self.train_dataloader()))
        for img in x:
            self.logger.experiment.log_image(
                img, image_channels="first", image_minmax=(0.0, 1.0)
            )
        for img in y:
            self.logger.experiment.log_image(
                img, image_channels="first", image_minmax=(0.0, 1.0)
            )
            
    def on_epoch_start(self):
        for tfm in self.trainer.datamodule.data.valid.tfms:
            if "Deterministic" in str(type(tfm)):
                tfm.n = 0

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.color_converter(x).detach()
        return self.unet(x)

    def show_results(
        self, ds: Dataset, n: int = 16, imgsize: int = 4, title: Optional[str] = None
    ):
        """
        Plot `n` predictions from the normalizer using samples from dataset
        `ds`. Each line will contain input, target and prediction images (in
        that order).
        """
        n = min(n, self.bs)
        fig, axs = plt.subplots(n, 3, figsize=(imgsize * 3, imgsize * n))
        idxs = np.random.choice(np.arange(len(ds)), size=n, replace=False)
        inputs = []
        targs = []
        for idx in idxs:
            x, y = ds[idx]
            inputs.append(x)
            targs.append(y)
        inputs = torch.stack(inputs).to(next(self.main_device))
        preds = self.predict(inputs).clamp(0, 1)
        for ax_r, x, y, z in zip(axs, inputs, targs, preds):
            x = x.cpu().numpy().transpose(1, 2, 0)
            y = y.numpy().transpose(1, 2, 0)
            z = z.detach().cpu().numpy().transpose(1, 2, 0)
            show_img(x, ax=ax_r[0])
            show_img(y, ax=ax_r[1])
            show_img(z, ax=ax_r[2])
        title = ifnone(title, "input/target/prediction")
        fig.suptitle(title)
        plt.show()

    def freeze_encoder(self):
        """
        Freeze the encoder part of the normalizer.
        """
        for m in self.leaf_modules:
            if "encoder" in m.name and not isinstance(m, nn.BatchNorm2d):
                for param in m.parameters():
                    param.requires_grad = False

    def init_bn(self):
        """
        Initialize BatchNorm layers with bias `1e-3` and weights `1`.
        """
        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d):
                with torch.no_grad():
                    m.bias.fill_(1e-3)
                    m.weight.fill_(1.0)

    def validation_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_nb: int
    ) -> Dict[str, torch.Tensor]:
        # OPTIONAL
        ret = super().validation_step(batch, batch_nb)
        y, y_hat = ret.pop("labels"), ret.pop("preds")
        bs = y.shape[0]
        y = rgb_to_lab(y.detach())
        y_hat = rgb_to_lab(y_hat.detach())
        ret["mu_x"] = torch.mean(y[:, 0], axis=(1, 2)).float()
        ret["sigma_x"] = torch.std(y[:, 0], axis=(1, 2)).float()
        ret["mu_y"] = torch.mean(y_hat[:, 0], axis=(1, 2)).float()
        ret["sigma_y"] = torch.std(y_hat[:, 0], axis=(1, 2)).float()
        ret["mu_xy"] = torch.mean(y[:, 0] * y_hat[:, 0], axis=(1, 2)).float()
        return ret

    def validation_epoch_end(self, outputs: List[Dict[str, torch.Tensor]]):
        # OPTIONAL
        super().validation_epoch_end(outputs)
        log = {}
        mu_x, sigma_x, mu_y, sigma_y, mu_xy = _get_keys_in_list_and_apply(
            outputs, "mu_x", "sigma_x", "mu_y", "sigma_y", "mu_xy", apply_func=torch.cat
        )
        m_ssim = ssim(mu_x, sigma_x, mu_y, sigma_y, mu_xy)
        m_pcc = pcc(mu_x, sigma_x, mu_y, sigma_y, mu_xy)
        m_cd = sigma_y / mu_y - sigma_x / mu_x
        log["ssim"] = m_ssim.mean()
        log["pcc"] = m_pcc.mean()
        log["cd"] = m_cd.mean()
        self.log_dict(log)

    def test_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_nb: int
    ) -> Dict[str, torch.Tensor]:
        return self.validation_step(batch, batch_nb)

    def test_epoch_end(self, outputs: List[Dict[str, torch.tensor]]):
        return self.validation_epoch_end(outputs)

Module containing a UNet normalizer. Takes grayscale images as inputs and their corresponding colored version as outputs.

In [ ]:
show_doc(Normalizer.show_results)

<h4 id="Normalizer.show_results" class="doc_header"><code>Normalizer.show_results</code><a href="__main__.py#L36" class="source_link" style="float:right">[source]</a></h4>

> <code>Normalizer.show_results</code>(**`ds`**:`Dataset`, **`n`**:`int`=*`16`*, **`imgsize`**:`int`=*`4`*, **`title`**:`Optional`\[`str`\]=*`None`*)

Plot `n` predictions from the normalizer using samples from dataset
`ds`. Each line will contain input, target and prediction images (in
that order).

In [ ]:
show_doc(Normalizer.freeze_encoder)

<h4 id="Normalizer.freeze_encoder" class="doc_header"><code>Normalizer.freeze_encoder</code><a href="__main__.py#L66" class="source_link" style="float:right">[source]</a></h4>

> <code>Normalizer.freeze_encoder</code>()

Freeze the encoder part of the normalizer.

In [ ]:
show_doc(Normalizer.init_bn)

<h4 id="Normalizer.init_bn" class="doc_header"><code>Normalizer.init_bn</code><a href="__main__.py#L75" class="source_link" style="float:right">[source]</a></h4>

> <code>Normalizer.init_bn</code>()

Initialize BatchNorm layers with bias `1e-3` and weights `1`.

In [ ]:
# export
class ClassifModule(BaseModule):
    def __init__(
        self,
        n_classes: int = 2,
        classes: Optional[Sequence[str]] = None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.save_hyperparameters()
        hparams = self.hparams
        hparams.classes = ifnone(classes, list(range(n_classes)))
        hparams.n_classes = len(
            hparams.classes
        )  # if classes is specified it superseeds n_classes
        self.loss = _get_loss(
            hparams.loss,
            hparams.weights,
            hparams.reduction,
            device=self.main_device,
            nc=n_classes,
        )
        self.cm = ConfusionMatrix(hparams.n_classes, compute_on_step=False)
        self.roc = ROC(num_classes=hparams.n_classes, compute_on_step=False)
        self.metrics = ClassifMetrics(
            n_classes=hparams.n_classes, compute_on_step=False
        )
        self._create_normalizer()

    def on_fit_start(self):
        x, _ = next(iter(self.train_dataloader()))
        if hasattr(self, "norm"):
            x = x.to(self.main_device)
            x = self.norm(x).clamp(0, 1).detach().cpu()
        for img in x:
            self.logger.experiment.log_image(
                img, image_channels="first", image_minmax=(0.0, 1.0)
            )

    def log_distribution(self, y: torch.Tensor):
        log = {}
        for k, cl in enumerate(self.hparams.classes):
            log[f"n_{cl}"] = (y == k).float().sum()
        self.log_dict(log, on_step=False, on_epoch=True, reduce_fx=torch.sum)

    def training_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_nb: int
    ) -> Dict[str, torch.Tensor]:
        ret = super().training_step(batch, batch_nb)
        _, y = batch
        self.log_distribution(y)
        return ret

    def update_metrics(self, y_hat: torch.Tensor, y: torch.Tensor):
        y = y.to(self.main_device)
        y_hat = y_hat.to(self.main_device)
        self.roc(y_hat, y)
        self.cm(y_hat, y)
        self.metrics(y_hat, y)

    def validation_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_nb: int
    ) -> Dict[str, torch.Tensor]:
        # OPTIONAL
        ret = super().validation_step(batch, batch_nb)
        y, y_hat = ret["labels"], ret["preds"]
        y_hat = torch.softmax(y_hat, 1, torch.float)
        self.update_metrics(y_hat, y)
        ret["preds"] = y_hat
        return ret

    def log_metrics(self, metrics, cm, roc, suffix: str = None):
        log = {}
        app = f"_{suffix}" if suffix is not None else ""
        metric_dict = metrics.compute()
        for metric in metric_dict:
            val = metric_dict[metric]
            if val.numel() == 1:
                log[metric+app] = val
            else:
                for k, cl in enumerate(self.hparams.classes):
                    c_name = f"{metric}{app}_{cl}"
                    log[c_name] = val[k]
                log[f"{metric}{app}_mean"] = val.mean()
        if not self.trainer.running_sanity_check:
            mat = cm.compute().cpu().numpy()
            self.logger.experiment.log_confusion_matrix(
                labels=self.hparams.classes,
                matrix=mat,
                step=self.global_step,
                epoch=self.current_epoch,
                file_name=f"confusion_matrix{app}_{self.current_epoch}.json",
            )
            fprs, tprs, _ = roc.compute()
            for cl, fpr, tpr in zip(self.hparams.classes[::-1], fprs[::-1], tprs[::-1]):
                fpr = fpr.cpu().numpy()
                tpr = tpr.cpu().numpy()
                self.logger.experiment.log_curve(
                    f"ROC{app}_{cl}_{self.current_epoch}",
                    x=fpr.tolist(),
                    y=tpr.tolist(),
                    step=self.current_epoch,
                    overwrite=False,
                )
                log[f"AUC{app}_{cl}"] = auc(fpr, tpr)
                if self.hparams.n_classes == 2:
                    break
        metrics.reset()
        cm.reset()
        roc.reset()
        self.log_dict(log, on_step=False, on_epoch=True)

    def log_preds(self, preds: torch.Tensor, labels: torch.Tensor):
        preds = preds.numpy()
        labels = labels.numpy()
        items = self.trainer.datamodule.data.valid.items.astype(str)
        data = {
            f"pred_{self.hparams.classes[k]}": preds[:, k]
            for k in range(preds.shape[1])
        }
        data["item"] = items[: labels.shape[0]]
        data["label"] = labels
        keys = ["item", "label"] + [f"pred_{cl}" for cl in self.hparams.classes]
        df = pd.DataFrame(data=data, columns=keys)
        savepath = self.save_path / f"lightning_logs/version_{self.version}/preds"
        if not savepath.is_dir():
            savepath.mkdir()
        df.to_csv(savepath / f"{self.current_epoch}.csv", index=False)

    def log_slide_metrics(self, preds: torch.Tensor, labels: torch.Tensor):
        if not self.trainer.running_sanity_check:
            items = self.trainer.datamodule.data.valid.items
            patch_slides = np.vectorize(lambda x: x.parent.name)(items)
            slides = np.unique(patch_slides)
            slide_labels = []
            slide_preds = []
            roc = ROC(num_classes=self.hparams.n_classes, compute_on_step=False)
            cm = ConfusionMatrix(self.hparams.n_classes, compute_on_step=False)
            metrics = ClassifMetrics(
                n_classes=self.hparams.n_classes, compute_on_step=False
            )
            for slide in slides:
                idxs = np.argwhere(patch_slides == slide).squeeze()
                label = labels[idxs][0]
                pred = preds[idxs].mean(0)
                slide_labels.append(label)
                slide_preds.append(pred)
            slide_labels = torch.stack(slide_labels)
            slide_preds = torch.stack(slide_preds)
            cm(slide_preds, slide_labels)
            metrics(slide_preds, slide_labels)
            roc(slide_preds, slide_labels)
            self.log_metrics(metrics, cm, roc, suffix="slide")

    def validation_epoch_end(self, outputs: List[Dict[str, torch.Tensor]]):
        # OPTIONAL
        super().validation_epoch_end(outputs)
        self.log_metrics(self.metrics, self.cm, self.roc)
        preds, labels = _get_keys_in_list_and_apply(
            outputs, "preds", "labels", apply_func=torch.cat
        )
        self.log_preds(preds, labels)

    def predict(self, x: torch.Tensor) -> torch.Tensor:
        pred = super().predict(x)
        pred = torch.softmax(pred, dim=1)
        return pred

In [ ]:
# export
class ImageClassifModel(ClassifModule):
    """"""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        hparams = self.hparams
        in_chans = 3 if hparams.transforms < 10 else 9
        self.base_model = _get_base_model(
            hparams.model,
            pretrained=not hparams.rand_weights,
            in_chans=in_chans,
            size=hparams.size,
        )
        nf = get_num_features(self.base_model)
        self.head = _get_head(nf, hparams.n_classes, dropout=hparams.dropout)
        self.post_init()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            if hasattr(self, "norm"):
                x = self.norm(x)
            x = self.color_converter(x)
        x = self.base_model(x)
        x = self.head(x)
        return x

    def freeze_base(self):
        """
        Freeze the base model.
        """
        for m in self.leaf_modules:
            if "base_model" in m.name and not isinstance(m, nn.BatchNorm2d):
                for param in m.parameters():
                    param.requires_grad = False
                    
    def validation_epoch_end(self, outputs: List[Dict[str, torch.Tensor]]):
        super().validation_epoch_end(outputs)
        self.log_slide_metrics(preds, labels)

In [ ]:
# export
class RNNAttention(ClassifModule):
    """"""

    def __init__(
        self,
        n_glimpses: int = 3,
        glimpse_size: int = 256,
        gamma: float = 1,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.save_hyperparameters()
        hparams = self.hparams
        nc = hparams.n_classes
        self.t_x = nn.Sequential(*list(CBR(3, 64, 2).children())[:-2])
        nf = get_num_features(self.t_x)
        self.flat = nn.Flatten()
        self.fc = nn.Linear(nf, nc)
        self.t_l = nn.Sequential(
            nn.Linear(6, nf),
            nn.ReLU(),
            nn.Linear(nf, 2 * nf),
            nn.ReLU(),
            nn.Linear(2 * nf, nf),
        )
        self.t_a = nn.Sequential(
            nn.Linear(nf, 2 * nf),
            nn.ReLU(),
            nn.Linear(2 * nf, nf),
            nn.ReLU(),
            nn.Linear(nf, 6),
        )
        self.final_head = nn.Sequential(
            nn.Linear(nf * n_glimpses, nf), nn.ReLU(), nn.Linear(nf, nc)
        )

    def forward(
        self, X: torch.Tensor, l: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        with torch.no_grad():
            if hasattr(self, "norm"):
                X = self.norm(X)
            X = self.color_converter(X)
        Ai = gaussian_mask(
            l[:, 0], l[:, 1], l[:, 2], self.hparams.glimpse_size, self.hparams.size
        )
        Aj = gaussian_mask(
            l[:, 3], l[:, 4], l[:, 5], self.hparams.glimpse_size, self.hparams.size
        )
        x = Ai[:, None] @ X @ Aj.transpose(1, 2)[:, None]
        fx = self.flat(self.t_x(x))
        y = self.fc(fx)
        fl = self.t_l(l)
        l = torch.sigmoid(fx * fl)
        l = self.t_a(l)
        return fx, y, l

    def compute_loss(
        self, X: torch.Tensor, Y: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Compute the loss defined in the paper. Return the final slide-level
        prediction as well.
        """
        l0 = torch.tensor(
            [[0, 0.5, self.hparams.size // self.hparams.glimpse_size] * 2] * X.shape[0],
            device=self.main_device,
        )
        loss = 0
        loss_prev = 0
        preds = []
        fts = []
        for t in range(self.hparams.n_glimpses):
            fx, y_hat, l = self(X, l0)
            fts.append(fx)
            loss += self.loss(y_hat, Y)
            loss_a = ((torch.softmax(y_hat, 1, torch.float) - 0.5) ** 2).mean()
            if t > 0:
                loss -= loss_a - loss_prev / t
            loss_prev += loss_a
            loss += (self.hparams.gamma * torch.exp(-torch.abs(l - l0))).mean()
            l0 = l.detach()
        # bs x (C x n_glimpses)
        fts = torch.cat(fts, dim=1)
        Y_hat = self.final_head(fts)
        loss += self.loss(Y_hat, Y)
        return loss, Y_hat

    def training_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_nb: int
    ) -> Dict[str, torch.Tensor]:
        # REQUIRED
        X, Y = batch
        loss, _ = self.compute_loss(X, Y)
        lr = self.opt.param_groups[-1]["lr"]
        log = {"train_loss": loss, "learning_rate": lr}
        self.log_dict(log)
        self.log_distribution(Y)
        return {"loss": loss}

    def validation_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_nb: int
    ) -> Dict[str, torch.Tensor]:
        # OPTIONAL
        x, y = batch
        loss, y_hat = self.compute_loss(x, y)
        n = y.shape[0]
        y_hat = torch.softmax(y_hat, 1, torch.float)
        self.update_metrics(y_hat, y)
        return ret

RNN model with attention as defined in [_Predicting Cancer with a Recurrent Visual Attention Model for Histopathology Images_](http://www.sfu.ca/~abentaie/papers/miccai18.pdf). 

In [ ]:
show_doc(RNNAttention.compute_loss)

<h4 id="RNNAttention.compute_loss" class="doc_header"><code>RNNAttention.compute_loss</code><a href="__main__.py#L58" class="source_link" style="float:right">[source]</a></h4>

> <code>RNNAttention.compute_loss</code>(**`X`**:`Tensor`, **`Y`**:`Tensor`)

Compute the loss defined in the paper. Return the final slide-level
prediction as well.

In [ ]:
# export
def get_topk_idxs(probs, slide_idxs, k=1):
    order = np.lexsort((probs, slide_idxs))
    probs = probs[order]
    slide_idxs = slide_idxs[order]
    index = np.empty(len(probs), 'bool')
    index[-k:] = True
    index[:-k] = slide_idxs[k:] != slide_idxs[:-k]        
    idxs = order[index]
    return idxs

In [ ]:
# export
def get_max_probs(probs, slide_idxs):
    nmax = slide_idxs.max()
    out = np.empty(nmax+1)
    out[:] = np.nan
    order = np.lexsort((probs, slide_idxs))
    probs = probs[order]
    slide_idxs = slide_idxs[order]
    index = np.empty(len(probs), 'bool')
    index[-1] = True
    index[:-1] = slide_idxs[1:] != slide_idxs[:-1]
    out[slide_idxs[index]] = probs[index]
    return out

In [ ]:
# export
class MILModel(ImageClassifModel):
    """"""
    def configure_optimizers(self):
        opt = super().configure_optimizers()
        if isinstance(opt, dict) and "lr_scheduler" in opt:
            n_slides = len(self.trainer.datamodule.data.test.slides)
            n_steps = (n_slides * self.hparams.topk) // self.hparams.batch_size
            opt["lr_scheduler"] = _get_scheduler(
                opt["optimizer"],
                self.hparams.sched,
                n_steps * self.hparams.epochs,
                self.hparams.lr,
            )
            self.sched = opt["lr_scheduler"]["scheduler"]
        return opt

    def on_train_epoch_start(self):
        self.trainer.verbose_test = False
        k = self.hparams.topk
        self.trainer.testting = True
        probs = _get_keys_in_list_and_apply(
            self.trainer.run_evaluation(test_mode=True)[0],
            "probs",
            apply_func=torch.cat,
        )[0]
        self.trainer.testing = False
        probs = probs.cpu().numpy()
        test_ds = self.trainer.datamodule.data.test
        slide_idxs = test_ds.slide_idxs
        idxs = get_topk_idxs(probs, slide_idxs, k=k)
        train_ds = self.trainer.datamodule.data.train
        train_ds.items = test_ds.items.copy()[idxs]
        train_ds.labels = test_ds.labels.copy()[idxs]
        train_ds.slide_idxs = test_ds.slide_idxs.copy()[idxs]
        self.trainer.reset_train_dataloader(self)

    def validation_epoch_end(self, outputs: List[Dict[str, torch.Tensor]]):
        # OPTIONAL
        probs = _get_keys_in_list_and_apply(outputs, "preds", apply_func=torch.cat)[0][:, 1]
        loss = _get_keys_in_list_and_apply(outputs, "val_loss", apply_func=torch.stack)[
            0
        ]
        probs = probs.cpu().numpy()
        log = {"val_loss": loss.mean()}
        val_ds = self.trainer.datamodule.data.valid
        preds = torch.as_tensor(get_max_probs(probs, val_ds.slide_idxs))
        targs = torch.tensor(
            [val_ds.label_loader(label) for label in val_ds.slide_labels]
        )
        self.metrics.reset()
        self.cm.reset()
        self.roc.reset()
        self.update_metrics(preds, targs)
        self.log_metrics(self.metrics, self.cm, self.roc)

    def test_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_nb: int
    ) -> Dict[str, torch.Tensor]:
        return self.validation_step(batch, batch_nb)

In [ ]:
# export
class RNNAggregator(ImageClassifModel):
    def __init__(self, rnn_hidden_dims: int = 128, **kwargs):
        super().__init__(**kwargs)
        self.cpu()
        self.freeze()
        nf = get_num_features(self.base_model)
        self.pooling = nn.Sequential(nn.AdaptiveAvgPool2d(1), nn.Flatten())
        self.fc1 = nn.Linear(nf, rnn_hidden_dims)
        self.fc2 = nn.Linear(rnn_hidden_dims, rnn_hidden_dims)
        self.fc3 = nn.Linear(rnn_hidden_dims, self.hparams.n_classes)
        self.act = nn.ReLU()
        self.example_inputs = (
            self.example_inputs,
            torch.zeros(2, rnn_hidden_dims, device=self.main_device),
        )
        self.save_hyperparameters()
        self = self.to(self.main_device)

    def post_init(self):
        self = self.to(self.main_device)
        if self.hparams.resume_version is not None:
            self.load(
                self.hparams.resume_version, ckpt_epoch=self.hparams.resume_checkpoint
            )

    def forward(self, x, state):
        with torch.no_grad():
            if hasattr(self, "norm"):
                x = self.norm(x)
            x = self.color_converter(x)
        x = self.base_model(x)
        x = self.fc1(self.pooling(x))
        state = self.fc2(state)
        state = self.act(state + x)
        y = self.fc3(state)
        return y, state

    def on_train_epoch_start(self):
        self.base_model.eval()

    def training_step(
        self, batch: Tuple[List[torch.Tensor], torch.Tensor], batch_nb: int
    ):
        xs, y = batch
        self.log_distribution(y)
        state = torch.zeros(
            y.shape[0], self.hparams.rnn_hidden_dims, device=self.main_device
        )
        for x in xs:
            y_hat, state = self(x, state)
        loss = self.loss(y_hat, y)
        lr = self.opt.param_groups[-1]["lr"]
        log = {"train_loss": loss.detach(), "learning_rate": lr}
        self.log_dict(log, on_step=True, on_epoch=False)
        return loss

    def validation_step(
        self, batch: Tuple[List[torch.Tensor], torch.Tensor], batch_nb: int
    ) -> Dict[str, torch.Tensor]:
        # OPTIONAL
        xs, y = batch
        n = y.shape[0]
        state = torch.zeros(n, self.hparams.rnn_hidden_dims, device=self.main_device)
        for x in xs:
            y_hat, state = self(x, state)
        loss = self.loss(y_hat, y).detach()
        ret = {"loss": loss}
        y_hat = torch.softmax(y_hat, 1, torch.float)
        y_hat = y_hat.argmax(dim=-1).view(n, -1)
        y = y.view(n, -1)
        for k, cl in enumerate(self.hparams.classes):
            ret[f"t_{cl}"] = ((y_hat == k) & (y == k)).float().sum()
            ret[f"f_{cl}"] = ((y_hat == k) & (y != k)).float().sum()
        return ret

    def predict(self, xs: List[torch.Tensor]) -> torch.Tensor:
        state = torch.zeros(x.shape[0], self.hparams.rnn_hidden_dims)
        for x in xs:
            pred, state = super().predict(x, state)
        pred = torch.softmax(pred, dim=1)
        return pred

In [ ]:
# export
class CoTeachingModel(ClassifModule):
    """"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        hparams = self.hparams
        self.loss = _get_loss(
            hparams.loss, hparams.weights, "none", device=self.main_device, nc=n_classes
        )
        base_model1 = _get_base_model(
            hparams.model, pretrained=not hparams.rand_weights
        )
        base_model2 = _get_base_model(
            hparams.model, pretrained=not hparams.rand_weights
        )
        nf = get_num_features(base_model1)
        head1 = _get_head(nf, n_classes, dropout=hparams.dropout)
        head2 = _get_head(nf, n_classes, dropout=hparams.dropout)
        self.model1 = Classifier(base_model1, head1)
        self.model2 = Classifier(base_model2, head2)
        self.model2.apply(nn.init.kaiming_normal_)
        self.cms = [
            ConfusionMatrix(n_classes, compute_on_step=False),
            ConfusionMatrix(n_classes, compute_on_step=False),
        ]
        self.rocs = [
            ROC(num_classes=n_classes, compute_on_step=False),
            ROC(num_classes=n_classes, compute_on_step=False),
        ]
        self.metrics = [
            ClassifMetrics(n_classes=n_classes, compute_on_step=False),
            ClassifMetrics(n_classes=n_classes, compute_on_step=False),
        ]
        self.post_init()

    def configure_optimizers(self) -> Union[Optimizer, Dict[str, Any]]:
        # REQUIRED
        hparams = self.hparams
        if not hasattr(self, "loss"):
            self.loss = _get_loss(
                hparams.loss,
                hparams.weights,
                hparams.reduction,
                device=self.main_device,
            )
        self.lr = hparams.lr
        self.wd = hparams.wd
        self.opt1 = torch.optim.AdamW(
            self.model1.parameters(), lr=self.lr, weight_decay=self.wd
        )
        self.opt1 = torch.optim.AdamW(
            self.model2.parameters(), lr=self.lr, weight_decay=self.wd
        )
        n_train_dl = len(self.trainer.datamodule.train_dataloader())
        n_iter = int(hparams.epochs * n_train_dl)
        sched1 = _get_scheduler(self.opt1, hparams.sched, n_iter, self.lr)
        sched2 = _get_scheduler(self.opt2, hparams.sched, n_iter, self.lr)
        if sched1 is None:
            return self.opt1, self.opt2
        else:
            self.sched1 = sched1["scheduler"]
            self.sched2 = sched2["scheduler"]
            return [self.opt1, self.opt2], [self.sched1, self.sched2]

    def on_train_epoch_start(self):
        epoch = self.trainer.current_epoch
        epoch_max = self.trainer.max_epoch + 1
        eps = self.hparams.forget_rate
        n = self.hparams.forget_steps
        self.remember_rate = 1 - eps * min(1 + (epoch - n) / (epoch_max - n), epoch / n)

    def get_losses(self, batch: Tuple[torch.Tensor, torch.Tensor]):
        x, y = batch
        self.log_distribution(y)
        num_remember = int(self.remember_rate * len(y))
        y1 = self(x, model_idx=1)
        y2 = self(x, model_idx=2)
        loss1 = self.loss(y1, y).detach()
        loss2 = self.loss(y2, y).detach()

        if self.hparams.disagree_only:
            pred1 = torch.softmax(y1, 1, torch.float).argmax(dim=1).detach()
            pred2 = torch.softmax(y1, 1, torch.float).argmax(dim=1).detach()
            disagree_idxs = (pred1 != pred2).cpu().data
            idx1_sorted = np.lexsort((loss1.cpu().data, ~disagree_idxs))
            idx2_sorted = np.lexsort((loss2.cpu().data, ~disagree_idxs))
        else:
            idx1_sorted = torch.argsort(loss1)
            idx2_sorted = torch.argsort(loss2)

        y1 = y1[idx2_sorted[:num_remember]]
        y2 = y2[idx1_sorted[:num_remember]]
        loss1 = self.loss(y1, y[idx2_sorted[:num_remember]]).mean()
        loss2 = self.loss(y2, y[idx2_sorted[:num_remember]]).mean()
        return loss1, loss2, y1.detach(), y2.detach()

    def training_step(
        self,
        batch: Tuple[torch.Tensor, torch.Tensor],
        batch_nb: int,
        optimizer_idx: int,
    ) -> torch.Tensor:
        # REQUIRED
        loss1, loss2, _, _ = self.get_losses(batch)
        opt1, opt2 = self.optimizers()
        self.manual_backward(loss1, opt1)
        self.manual_backward(loss2, opt2)
        self.manual_optimizer_step(opt1)
        self.manuam_optimizer_step(opt2)
        lr = self.opt.param_groups[-1]["lr"]
        log = {
            "train_loss1": loss1.detach(),
            "train_loss2": loss2.detach(),
            "learning_rate": lr,
        }
        self.log_dict(log, on_step=True, on_epoch=False)

    def validation_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_nb: int
    ) -> Dict[str, torch.Tensor]:
        # OPTIONAL
        x, y = batch
        y1 = self(x, model_idx=1)
        y2 = self(x, model_idx=2)
        loss1 = self.loss(y1, y).detach()
        loss2 = self.loss(y2, y).detach()
        self.log("loss_1", loss1, on_step=False, on_epoch=True)
        self.log("loss_2", loss2, on_step=False, on_epoch=True)
        y1 = torch.softmax(y1, 1, torch.float)
        y2 = torch.softmax(y2, 1, torch.float)
        ret = {"labels": y, "preds1": y1, "preds2": y2}
        for metrics, cm, roc, y_hat in zip(self.metrics, self.cms, self.rocs, (y1, y2)):
            roc(y_hat, y)
            metrics(y_hat, y)
            cm(y_hat, y)
        return ret

    def validation_epoch_end(self, outputs: List[Dict[str, torch.Tensor]]):
        # OPTIONAL
        for n, metrics, cm, roc in zip((1, 2), self.metrics, self.cms, self.epochs):
            self.log_metrics(metrics, cm, roc, suffix=n)

    def forward(self, x: torch.Tensor, model_idx=1) -> torch.Tensor:
        with torch.no_grad():
            if hasattr(self, "norm"):
                x = self.norm(x)
            x = self.color_converter(x)
        x = getattr(self, f"model{model_idx}")(x)
        return x

    def predict(self, x: torch.Tensor) -> torch.Tensor:
        pred = super().predict(x)
        pred = torch.softmax(pred, dim=1)
        return pred

    def freeze_base(self):
        """
        Freeze the base model.
        """
        for m in self.leaf_modules:
            if "base_model" in m.name and not isinstance(m, nn.BatchNorm2d):
                for param in m.parameters():
                    param.requires_grad = False

    def fit(self, dm: pl.LightningDataModule, **kwargs):
        super().fit(dm, automatic_optimization=False, **kwargs)

In [ ]:
# export
class WSITransformer(ClassifModule):
    def __init__(self, embed_dim=2048, **kwargs):
        super().__init__(**kwargs)
        params = self.hparams.model.split("_")
        assert params[0] == "vit", "model must be VIT for Transformer."
        assert (
            len(params) == 4
        ), "model must be formatted as vit_{num_heads}_{mlp_ratio}_{depth}"
        num_heads, mlp_ratio, depth = map(int, params[1:])
        self.blocks = nn.ModuleList(
            [
                Block(
                    dim=embed_dim,
                    num_heads=num_heads,
                    mlp_ratio=mlp_ratio,
                    drop=self.hparams.dropout,
                    act_layer=nn.ReLU,
                    norm_layer=nn.GroupNorm
                )
                for i in range(depth)
            ]
        )
        self.norm = nn.LayerNorm(embed_dim, eps=1e-6)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(
            torch.zeros(1, self.hparams.size * self.hparams.size + 1, embed_dim)
        )
        self.pos_drop = nn.Dropout(p=self.hparams.dropout)
        self.head = nn.Linear(embed_dim, self.hparams.n_classes)
        trunc_normal_(self.pos_embed)
        trunc_normal_(self.cls_token)
        self.apply(self._init_weights)
        self.save_hyperparameters()
        self.post_init()

    def post_init(self):
        leaf_modules = named_leaf_modules(self)
        size = self.hparams.size
        self.sizes, self.leaf_modules = get_sizes(
            self,
            input_shape=(size * size, self.hparams.embed_dim),
            leaf_modules=leaf_modules,
        )
        self = self.to(self.main_device)
        if self.hparams.resume_version is not None:
            self.load(
                self.hparams.resume_version, ckpt_epoch=self.hparams.resume_checkpoint
            )

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=0.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward_features(self, x):
        B = x.shape[0]
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)
        for block in self.blocks:
            x = block(x)
        x = self.norm(x)[:, 0]
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x

In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_train.ipynb.
Converted 02_predict.ipynb.
Converted 10_data.read.ipynb.
Converted 11_data.loaders.ipynb.
Converted 12_data.dataset.ipynb.
Converted 13_data.utils.ipynb.
Converted 14_data.transforms.ipynb.
Converted 15_data.color.ipynb.
Converted 16_data.modules.ipynb.
Converted 20_models.plmodules.ipynb.
Converted 21_models.modules.ipynb.
Converted 22_models.utils.ipynb.
Converted 23_models.hooks.ipynb.
Converted 24_models.metrics.ipynb.
Converted 25_models.losses.ipynb.
Converted 80_params.defaults.ipynb.
Converted 81_params.parser.ipynb.
Converted 99_index.ipynb.
